In [ ]:
# Only needed if working in Colab
#!git clone https://github.com/sprouse9/URI_CapstoneProject.git

In [6]:
import os
import zipfile
import importlib.util
import subprocess

zip_filename = "CarDetectionDataSet.zip"
extract_folder = "archive"  # Define the folder where files are extracted

print(f"Working dir: {os.getcwd()}")  # This shows your current working directory

# Download only if the zip file and extracted folder don't exist
if not os.path.exists(zip_filename) and not os.path.exists(extract_folder):
    # Check if gdown is installed before attempting to install
    if importlib.util.find_spec("gdown") is None:
        print("gdown not found. Installing...")
        !pip install gdown

    print(f"{zip_filename} not found. Downloading...")
    !gdown 1JFAfrbUfXtiF-xwko2ACB-snDwIsj31h -O {zip_filename}
else:
    print(f"Skipping download. {zip_filename} or {extract_folder} already exists.")

# Extract only if the extracted folder does not exist
if not os.path.exists(extract_folder):
    print(f"Extracting {zip_filename}...")
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall()
    print(f"Extraction complete: {extract_folder}")
else:
    print(f"Data folder (archive) already exists. Skipping extraction.")

Working dir: /Users/randy/DSP577/URI_CapstoneProject
Skipping download. CarDetectionDataSet.zip or archive already exists.
Extracting CarDetectionDataSet.zip...
Extraction complete: archive


In [ ]:
!pip install ultralytics

In [ ]:
import cv2
import matplotlib.pyplot as plt

#cv2.__version__

# Load an image (replace with your image path)
imagepath = "images/exampleframe.jpg"
image = cv2.imread(imagepath)

# Convert BGR to RGB (OpenCV loads images in BGR by default)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image using matplotlib
plt.imshow(image_rgb)
plt.axis('off')  # Turn off axis
plt.show()


In [ ]:
import torch
from ultralytics import YOLO

print("PyTorch Version:", torch.__version__)

model = YOLO("yolov8n.pt")

# model.to("cpu")

results = model(imagepath)
#print(results)

In [ ]:
import os

dataset_path = os.path.join(os.getcwd(), "archive/train/images")
print(f"Dataset path: {dataset_path}")
print("Exists?", os.path.exists(dataset_path))


In [ ]:
import os

# Print full path to make sure it's correct
train_images_path = os.path.join(os.getcwd(), "archive/train/images")
val_images_path = os.path.join(os.getcwd(), "archive/val/images")

print(f"Training images path: {train_images_path}")
print(f"Validation images path: {val_images_path}")

# Check if the images directories exist
print("Training images directory exists?", os.path.exists(train_images_path))
print("Validation images directory exists?", os.path.exists(val_images_path))


In [ ]:
import yaml
with open("dataset.yaml", "r") as f:
    data_config = yaml.safe_load(f)

print("Detected Dataset Path:", data_config.get("path"))
print("Absolute Path:", os.path.abspath(data_config.get("path", "")))


In [ ]:
os.getcwd()

In [ ]:
import yaml
import os

# Load and modify `data.yaml`
yaml_path = "dataset.yaml"
with open(yaml_path, "r", encoding="utf-8") as f:
    data_config = yaml.safe_load(f)

# Force YOLO to use the correct dataset path
dataset_root = os.path.abspath("archive").replace("\\", "/")  # Ensure absolute path with forward slashes
data_config["path"] = dataset_root  # Update `path` key

# Save modified YAML to a temp file
temp_yaml_path = "temp_data.yaml"
with open(temp_yaml_path, "w", encoding="utf-8") as f:
    yaml.dump(data_config, f, default_flow_style=False)

print(f"Updated dataset path: {dataset_root}")

# Train YOLO with the modified YAML





In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
results = model.train(data=temp_yaml_path, epochs=5,  batch=32, imgsz=640, project="runs/train");

In [ ]:
results.keys

In [ ]:
results.results_dict['metrics/precision(B)']

In [ ]:
results.results_dict

In [ ]:
dir(results)

In [ ]:
print(results.mean_results)

In [ ]:
print(results.results_dict)

# Let's print a summary

In [1]:
import pandas as pd
import torch
from ultralytics import YOLO

# Define hyperparameter grid
learning_rates = [0.0005, 0.002, 0.01] #, 0.005, 0.01]
epochs_list = [10, 20, 50] # , 15, 20, 25]

# List to store results from each experiment
results_summary = []

for lr in learning_rates:
    for epochs in epochs_list:
        print(f"Training with lr: {lr}, epochs: {epochs}")
        
        # Initialize the model
        model = YOLO("yolov8n.pt")
        
        # Train the model with the current hyperparameters
        results = model.train(
            data="temp_data.yaml",   # replace with your dataset YAML file path
            epochs=epochs,
            batch=16,
            imgsz=640,
            lr0=lr,
            #cache=True,
            cache='disk',
            optimizer="AdamW",
            project=f"runs/train/lr{lr}_ep{epochs}"
        )
        
        # Get the results dictionary
        rdict = results.results_dict  # Contains keys like 'metrics/precision(B)' etc.
        
        # Append the metrics along with the hyperparameters to our list
        results_summary.append({
            "lr": lr,
            "epochs": epochs,
            "precision": rdict.get("metrics/precision(B)", None),
            "recall": rdict.get("metrics/recall(B)", None),
            "mAP50": rdict.get("metrics/mAP50(B)", None),
            "mAP50-95": rdict.get("metrics/mAP50-95(B)", None),
            "fitness": rdict.get("fitness", None)
        })

        
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()


# Convert the results list into a DataFrame for easy viewing
results_df = pd.DataFrame(results_summary)
print("\nBaseline Metrics for Each Test:")
print(results_df)


Training with lr: 0.0005, epochs: 10
Ultralytics 8.3.75  Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8191MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=temp_data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=disk, device=None, workers=8, project=runs/train/lr0.0005_ep10, name=train4, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, 

train: Scanning C:\Users\RyzenGaming\source\repos\URI_CapstoneProject\archive\train\labels.cache... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/
train: Caching images (2.3GB Disk): 100%|██████████| 400/400 [00:00<00:00, 17375.99it/s]
val: Scanning C:\Users\RyzenGaming\source\repos\URI_CapstoneProject\archive\val\labels.cache... 99 images, 0 backgrounds, 0 corrupt: 100%|██████████| 99/99 [00
val: Caching images (0.6GB Disk): 100%|██████████| 99/99 [00:00<00:00, 19787.28it/s]


Plotting labels to runs\train\lr0.0005_ep10\train4\labels.jpg... 
optimizer: AdamW(lr=0.0005, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\train\lr0.0005_ep10\train4
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10       2.2G     0.9968      2.079     0.8673        173        640: 100%|██████████| 25/25 [00:05<00:00,  4.82it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.30it/s]

                   all         99        644     0.0109      0.505        0.4      0.251


...

      50/50      2.28G      0.368     0.2272     0.7803        176        640: 100%|██████████| 25/25 [00:03<00:00,  7.72it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  6.24it/s]

                   all         99        644      0.961      0.761       0.93      0.773



50 epochs completed in 0.076 hours.
Optimizer stripped from runs\train\lr0.01_ep50\train2\weights\last.pt, 6.2MB
Optimizer stripped from runs\train\lr0.01_ep50\train2\weights\best.pt, 6.2MB

Validating runs\train\lr0.01_ep50\train2\weights\best.pt...
Ultralytics 8.3.75  Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8191MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.86it/s]


                   all         99        644      0.961      0.761       0.93      0.773
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\train\lr0.01_ep50\train2

Baseline Metrics for Each Test:
       lr  epochs  precision    recall     mAP50  mAP50-95   fitness
0  0.0005      10   0.802202  0.807453  0.884780  0.666257  0.688109
1  0.0005      20   0.826621  0.799557  0.880337  0.680879  0.700825
2  0.0005      50   0.834352  0.821232  0.901091  0.715739  0.734274
3  0.0020      10   0.785226  0.823181  0.890468  0.679267  0.700387
4  0.0020      20   0.769922  0.858696  0.889744  0.729158  0.745216
5  0.0020      50   0.784961  0.872671  0.912198  0.763812  0.778651
6  0.0100      10   0.825189  0.756211  0.842662  0.672156  0.689207
7  0.0100      20   0.842392  0.771739  0.902446  0.735979  0.752626
8  0.0100      50   0.960709  0.760870  0.930202  0.773212  0.788911


In [2]:
import psutil
print(f"Available Memory: {psutil.virtual_memory().available / 1e9} GB")


Available Memory: 3.905482752 GB


In [3]:
import torch

if torch.cuda.is_available():
    gpu_id = torch.cuda.current_device()
    vram = torch.cuda.get_device_properties(gpu_id).total_memory / 1e9  # Convert bytes to GB
    free_vram = torch.cuda.memory_allocated(gpu_id) / 1e9  # Convert bytes to GB
    print(f"Total VRAM: {vram:.2f} GB")
    print(f"Used VRAM: {free_vram:.2f} GB")
    print(f"Available VRAM: {vram - free_vram:.2f} GB")
else:
    print("No GPU detected.")


Total VRAM: 8.59 GB
Used VRAM: 0.10 GB
Available VRAM: 8.49 GB
